# Vitis-AI Sample(Image Classification)

## Preparation

In [ ]:
import numpy as np
import cv2
import vitis_ai_library
import xir
from IPython.display import Image

# file path
MODEL_PATH = "inception_resnet_v2_tf.xmodel"
IMG_PATH = "test.jpg"
LABEL_PATH = "imagenet_label.txt"

# input size
WIDTH = 299
HEIGHT = 299

# label
str_lines = []
with open(LABEL_PATH, "rt") as rf:
    str_lines = rf.readlines()
labels = [x[:-1] for x in str_lines]

## Display Input Image

In [ ]:
Image(IMG_PATH)

## Load model and Create Instance

In [ ]:
%%time
g = xir.Graph.deserialize(MODEL_PATH)
runner = vitis_ai_library.GraphRunner.create_graph_runner(g)

## Prepare input/output buffers 

In [ ]:
# input buffer
inputDim = tuple(runner.get_inputs()[0].get_tensor().dims)
inputData = [np.empty(inputDim, dtype=np.int8)]

# resize input image
image = cv2.imread(IMG_PATH, cv2.IMREAD_COLOR)
image = cv2.resize(image, dsize=(WIDTH, HEIGHT))

# normalization
image = image/255.0

# quantization
fix_point = runner.get_input_tensors()[0].get_attr("fix_point")
scale = 2 ** fix_point
image = (image * scale).round()
image = image.astype(np.int8)

# BGR->RGB
image = image[:, :, [2, 1, 0]]
inputData[0][0] = image

# output buffer
outputData = runner.get_outputs()

## Prediction

In [ ]:
%%time
# prediction
job_id = runner.execute_async(inputData, outputData)
runner.wait(job_id)

## Confirmation of prediction results

In [ ]:
resultList = np.asarray(outputData[0])[0]
resultIdx = resultList.argmax()
resultVal = resultList[resultIdx]
print("Prediction : %s, %f"%(labels[resultIdx], resultVal))

## Delete Instance

In [ ]:
del runner